In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Load pickled data *Eurosat dataset
import pickle
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [9]:
# Reshapes raw array data to numpy array for both pixel data and label
X = np.array(X).reshape(-1, 64, 64, 3)
y = np.array(y)

In [10]:
# Image size based on Eurosat data, for RGB Channel
IMG_SIZE = 64
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [12]:
base_model = tf.keras.applications.resnet.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

94773248/94765736 [==============================] - 7s 0us/step


In [14]:
base_model.trainable = False
# Let's take a look at the base model architecture
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [15]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [16]:
prediction_layer = tf.keras.layers.Dense(1)

In [24]:
# forming the pretained model from Resnet50 trained on imagenet dataset
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
])

model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.add(tf.keras.layers.Activation('sigmoid'))

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2, 2, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                20490     
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [26]:
len(model.trainable_variables)

2

In [27]:
# Testing with pretained model, now able to obtained above 50% accuracy for unseen data.
# Will continue working on improving the underlying model classification accuracy.
model.fit(X, y, batch_size=32, epochs=3, validation_split=0.2)

Train on 21600 samples, validate on 5400 samples
Epoch 1/3
21600/21600 [==============================] - 358s 17ms/sample - loss: 2.1400 - accuracy: 0.6163 - val_loss: 2.1665 - val_accuracy: 0.4913
Epoch 2/3
21600/21600 [==============================] - 355s 16ms/sample - loss: 2.0526 - accuracy: 0.8145 - val_loss: 2.1548 - val_accuracy: 0.5133
Epoch 3/3
21600/21600 [==============================] - 353s 16ms/sample - loss: 2.0372 - accuracy: 0.8409 - val_loss: 2.1542 - val_accuracy: 0.5107
